<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/deeplabv3_1223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch torchvision pillow tqdm

In [4]:
!git clone https://github.com/VainF/DeepLabV3Plus-Pytorch.git
!cd DeepLabV3Plus-Pytorch
!pip install -r requirements.txt

Cloning into 'DeepLabV3Plus-Pytorch'...
remote: Enumerating objects: 705, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 705 (delta 25), reused 15 (delta 15), pack-reused 668 (from 2)
Receiving objects: 100% (705/705), 8.26 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (380/380), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [8]:
cd DeepLabV3Plus-Pytorch

/content/DeepLabV3Plus-Pytorch


In [9]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=c6503e22c03f1cea896a1b897a63d8666e0a4a9e1cbb247bb909de1057ddfd08
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [10]:
mkdir checkpoints

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!unzip /content/drive/MyDrive/UrbanASIF/anchor_Shenzhen.zip -d ./datasets/anchore_Shenzhen

Archive:  /content/drive/MyDrive/UrbanASIF/anchor_Shenzhen.zip
  inflating: ./datasets/anchore_Shenzhen/0.png  
  inflating: ./datasets/anchore_Shenzhen/1.png  
  inflating: ./datasets/anchore_Shenzhen/100.png  
  inflating: ./datasets/anchore_Shenzhen/1021.png  
  inflating: ./datasets/anchore_Shenzhen/106.png  
  inflating: ./datasets/anchore_Shenzhen/1062.png  
  inflating: ./datasets/anchore_Shenzhen/107.png  
  inflating: ./datasets/anchore_Shenzhen/1073.png  
  inflating: ./datasets/anchore_Shenzhen/108.png  
  inflating: ./datasets/anchore_Shenzhen/1087.png  
  inflating: ./datasets/anchore_Shenzhen/1110.png  
  inflating: ./datasets/anchore_Shenzhen/1111.png  
  inflating: ./datasets/anchore_Shenzhen/1166.png  
  inflating: ./datasets/anchore_Shenzhen/1175.png  
  inflating: ./datasets/anchore_Shenzhen/118.png  
  inflating: ./datasets/anchore_Shenzhen/1180.png  
  inflating: ./datasets/anchore_Shenzhen/119.png  
  inflating: ./datasets/anchore_Shenzhen/121.png  
  inflating: .

In [20]:
import os
import torch
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
import network.modeling  # 从 DeepLabV3Plus-Pytorch 导入

# 设置设备为 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"

# 定义模型参数
MODEL_NAME = 'deeplabv3plus_mobilenet'  # 可选: 'deeplabv3plus_resnet50', 'deeplabv3plus_mobilenet', etc.
NUM_CLASSES = 19  # Cityscapes 通常有19类
OUTPUT_STRIDE = 16  # 输出步幅，通常为 16

# 初始化模型
model = network.modeling.__dict__[MODEL_NAME](num_classes=NUM_CLASSES, output_stride=OUTPUT_STRIDE)

# 加载预训练权重
CHECKPOINT_PATH = './checkpoints/best_deeplabv3plus_mobilenet_cityscapes_os16.pth'  # 替换为实际路径
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state'])

# 移动模型到设备并设置为评估模式
model.to(device)
model.eval()

# 定义图像预处理
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),  # DeepLabV3+ 常用较大输入尺寸
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 均值
        std=[0.229, 0.224, 0.225]    # ImageNet 方差
    )
])

# 定义全局变量用于存储提取的特征
extracted_features = []

def hook_fn(module, input, output):
    """
    Hook 函数，将 ASPP 层的输出特征图进行全局平均池化和归一化。

    Args:
        module: 当前挂载 Hook 的模块
        input: 输入张量（未使用）
        output: 输出张量，即 ASPP 层的输出特征图 [B, 256, H', W']
    """
    with torch.no_grad():
        # 全局平均池化，沿 H' 和 W' 维度求平均
        features = output.mean(dim=[2, 3])  # [B, 256]
        # 归一化
        features = features / features.norm(dim=-1, keepdim=True)  # [B, 256]
        # 将特征移动到 CPU 并添加到全局列表
        extracted_features.append(features.cpu())

# 注册 Hook 到 ASPP 层（假设 ASPP 是 classifier[0]）
hook = model.classifier[0].register_forward_hook(hook_fn)

def extract_features_hooks(image_folder, output_file, model):
    global extracted_features
    extracted_features = []  # 重置特征列表

    # 获取并按文件名中的数字顺序排序图像路径
    image_paths = sorted(
        [
            os.path.join(image_folder, img)
            for img in os.listdir(image_folder)
            if img.lower().endswith(('.jpg', '.png'))
        ],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )

    # 处理每一张图像
    for image_path in tqdm(image_paths, desc=f"Processing {image_folder}"):
        try:
            # 打开图像并确保是RGB格式
            image = Image.open(image_path).convert("RGB")

            # 预处理
            inputs = preprocess(image).unsqueeze(0).to(device)  # [1, 3, 512, 512]

            # 前向传播获取特征
            with torch.no_grad():
                model(inputs)
                # Hook 已自动将特征添加到 `extracted_features`

            # 可选: 检查是否成功提取特征
            if len(extracted_features) == 0:
                print(f"No features extracted for {image_path}")

        except Exception as e:
            print(f"处理 {image_path} 时出错: {e}")

    # 移除 Hook
    hook.remove()

    # 将所有特征拼接成一个大的张量并保存
    if len(extracted_features) > 0:
        image_features = torch.cat(extracted_features, dim=0)  # [N, 256]
        torch.save(image_features, output_file)
        print(f"特征已保存到 {output_file}")
    else:
        print(f"在 {image_folder} 中未找到有效的图像")

# 调用特征提取函数
extract_features_hooks(
    image_folder='./datasets/anchor_Shenzhen',  # 替换为你的图像文件夹路径
    output_file='./Anchor/imgs_deeplabv3plus_ASPP_anchor.pt',  # 替换为你想保存的路径
    model=model
)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 195MB/s]
<ipython-input-20-14c190f986b0>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of t

TypeError: 'DeepLabHeadV3Plus' object is not subscriptable

In [21]:
print(model)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (low_level_features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        

In [22]:
print(model.classifier.aspp)

ASPP(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(320, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): ASPPConv(
      (0): Conv2d(320, 256, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): ASPPConv(
      (0): Conv2d(320, 256, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): ASPPConv(
      (0): Conv2d(320, 256, kernel_size=(3, 3), stride=(1, 1), padding=(18, 18), dilation=(18, 18), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2):

In [25]:
import os
import torch
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
import network.modeling  # 从 DeepLabV3Plus-Pytorch 导入

# 设置设备为 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"

# 定义模型参数
MODEL_NAME = 'deeplabv3plus_mobilenet'  # 可选: 'deeplabv3plus_resnet50', 'deeplabv3plus_resnet101', etc.
NUM_CLASSES = 19  # Cityscapes 通常有19类
OUTPUT_STRIDE = 16  # 输出步幅，通常为 16

# 初始化模型
model = network.modeling.__dict__[MODEL_NAME](num_classes=NUM_CLASSES, output_stride=OUTPUT_STRIDE)

# 加载预训练权重
CHECKPOINT_PATH = './checkpoints/best_deeplabv3plus_mobilenet_cityscapes_os16.pth'  # 替换为实际路径
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

# 加载 state_dict
model.load_state_dict(checkpoint['model_state'])

# 移动模型到设备并设置为评估模式
model.to(device)
model.eval()

# 定义图像预处理（与训练时保持一致）
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),  # DeepLabV3+ 常用较大输入尺寸
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 均值
        std=[0.229, 0.224, 0.225]    # ImageNet 方差
    )
])

# 定义全局变量用于存储提取的特征
extracted_features = []

def hook_fn(module, input, output):
    """
    Hook 函数，将 ASPP Project 层的输出特征图进行全局平均池化和归一化。

    Args:
        module: 当前挂载 Hook 的模块
        input: 输入张量（未使用）
        output: 输出张量，即 ASPP Project 层的输出特征图 [B, 256, H', W']
    """
    with torch.no_grad():
        # 全局平均池化，沿 H' 和 W' 维度求平均
        features = output.mean(dim=[2, 3])  # [B, 256]
        # 归一化
        features = features / features.norm(dim=-1, keepdim=True)  # [B, 256]
        # 将特征移动到 CPU 并添加到全局列表
        extracted_features.append(features.cpu())

# 注册 Hook 到 ASPP Project 层
# 根据模型结构，ASPP Project 层为 model.classifier.aspp.project
hook = model.classifier.aspp.project.register_forward_hook(hook_fn)

def extract_features_hooks(image_folder, output_file, model):
    global extracted_features
    extracted_features = []  # 重置特征列表

    # 获取并按文件名中的数字顺序排序图像路径
    # 假设文件名为纯数字，如 '1.jpg', '2.png', etc.
    try:
        image_paths = sorted(
            [
                os.path.join(image_folder, img)
                for img in os.listdir(image_folder)
                if img.lower().endswith(('.jpg', '.png'))
            ],
            key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
        )
    except ValueError:
        # 如果文件名包含非数字字符，按字母顺序排序
        image_paths = sorted(
            [
                os.path.join(image_folder, img)
                for img in os.listdir(image_folder)
                if img.lower().endswith(('.jpg', '.png'))
            ]
        )
        print("图像文件名包含非数字字符，已按字母顺序排序。")

    # 处理每一张图像
    for image_path in tqdm(image_paths, desc=f"Processing {image_folder}"):
        try:
            # 打开图像并确保是RGB格式
            image = Image.open(image_path).convert("RGB")

            # 预处理
            inputs = preprocess(image).unsqueeze(0).to(device)  # [1, 3, 512, 512]

            # 前向传播获取特征
            with torch.no_grad():
                model(inputs)
                # Hook 已自动将特征添加到 `extracted_features`

            # 可选: 检查是否成功提取特征
            if len(extracted_features) == 0:
                print(f"No features extracted for {image_path}")

        except Exception as e:
            print(f"处理 {image_path} 时出错: {e}")

    # 移除 Hook
    hook.remove()

    # 将所有特征拼接成一个大的张量并保存
    if len(extracted_features) > 0:
        image_features = torch.cat(extracted_features, dim=0)  # [N, 256]
        torch.save(image_features, output_file)
        print(f"特征已保存到 {output_file}")
    else:
        print(f"在 {image_folder} 中未找到有效的图像")

# 调用特征提取函数
extract_features_hooks(
    image_folder='./datasets/anchor_Shenzhen',  # 替换为你的图像文件夹路径
    output_file='./imgs_deeplabv3plus_ASPP_anchor.pt',  # 替换为你想保存的路径
    model=model
)


<ipython-input-25-59f743e796b3>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
Processing ./datasets/anchor_

特征已保存到 ./imgs_deeplabv3plus_ASPP_anchor.pt


In [27]:
# 加载保存的特征文件
image_features = torch.load('./imgs_deeplabv3plus_ASPP_anchor.pt')

# 打印特征的形状
print(f"特征的形状: {image_features.shape}")

特征的形状: torch.Size([143, 256])


<ipython-input-27-a601313eabfd>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_features = torch.load('./imgs_deeplabv3plus_ASPP_anchor.pt')
